<a href="https://colab.research.google.com/github/JDM-1609/Statistical-Process-Control-in-Injection-Machines/blob/main/SPC_Analysis_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXTRACCIÓN DE ESTADÍSTICOS INICIALES

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, Optional

# Lectura archivo del ALS

def read_als_summary(path: str, sheet_name: Optional[str] = None, decimal: Optional[str] = None) -> pd.DataFrame:
    # Carga base
    if path.lower().endswith((".xlsx", ".xls")):
        if sheet_name is None:
            xls = pd.ExcelFile(path)
            candidate = None
            for sh in xls.sheet_names:
                tmp = pd.read_excel(path, sheet_name=sh, header=None)
                mask = tmp.astype(str).apply(
                    lambda col: col.str.lower().str.strip().str.contains("valor nominal", na=False)
                ).any(axis=1)
                if mask.any():
                    candidate = sh
                    break
            if candidate is None:
                candidate = 0  # primera hoja como fallback
            df = pd.read_excel(path, sheet_name=candidate, header=None)
        else:
            df = pd.read_excel(path, sheet_name=sheet_name, header=None)
    else:
        # CSV (soporta coma decimal)
        kw = {}
        if decimal is not None:
            kw["decimal"] = decimal
            kw["thousands"] = "." if decimal == "," else None
        df = pd.read_csv(path, header=None, **kw)

    # Buscar el inicio del bloque (fila que contiene "Valor nominal")
    row_mask = df.astype(str).apply(
        lambda col: col.str.lower().str.strip().str.contains("valor nominal", na=False)
    ).any(axis=1)

    if not row_mask.any():
        raise ValueError("No encuentro la fila 'Valor nominal' en el archivo ALS (en esta hoja).")

    start = row_mask.idxmax()
    # Ventana suficientemente amplia de métricas
    sub = df.iloc[start:start+40].reset_index(drop=True)
    sub.columns = ["metric"] + [f"col_{i}" for i in range(1, sub.shape[1])]
    sub = sub.dropna(axis=1, how="all")
    return sub

# Normalización de métricas y extracción

def normalize_metric_name(s: str) -> str:
    s0 = str(s).lower().strip()
    aliases = {
        "valor nominal": "valor_nominal",
        "tolerancia inferior": "tol_inferior",
        "tolerancia superior": "tol_superior",
        "mínimo": "min", "minimo": "min",
        "máximo": "max", "maximo": "max",
        "xqq": "media_real",
        "rq": "range_rq",
        "sq": "range_sq",
        "sigma": "sigma",
        "cp": "cp",
        "cps": "cps",
        "cpi": "cpi",
        "cpd": "cpk_dir",
        "lisx": "lisx",
        "liix": "liix",
        "liss": "liss",
        "liis": "liis",
        "n": "n",
    }
    for k, v in aliases.items():
        if s0.startswith(k):
            return v
    return s0

def tidy_als_summary(sub: pd.DataFrame, rename_cols: Optional[Dict[str, str]] = None) -> pd.DataFrame:
    df = sub.copy()
    df["metric"] = df["metric"].astype(str).map(normalize_metric_name)

    var_cols = [c for c in df.columns if c != "metric"]
    if rename_cols:
        df = df.rename(columns=rename_cols)
        var_cols = [c for c in df.columns if c != "metric"]

    # Forzar numérico en lo que no es 'metric'
    for c in var_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Armar una fila por variable con todas las métricas como columnas
    rows = []
    for c in var_cols:
        r = df[["metric", c]].set_index("metric")[c]
        r.name = c
        rows.append(r)
    wide = pd.DataFrame(rows).reset_index(drop=False).rename(columns={"index": "variable"})
    return wide


# Construcción de la tabla de “Seguimiento inicial”
def make_tracking_table(
    wide: pd.DataFrame,
    cpk_objetivo: Optional[Dict[str, float]] = None,
    half_range: bool = False
) -> pd.DataFrame:
    """
    wide: salida de tidy_als_summary (una fila por variable con columnas:
          valor_nominal, tol_inferior, tol_superior, media_real, sigma, cp, cpk_dir, ...)
    cpk_objetivo: dict opcional por variable -> Cpk objetivo.
    half_range: si True usa (TolSuperior - Nominal); si False usa (TolSuperior - TolInferior).
    """
    out = wide.copy()

    # Rango inicial
    if half_range:
        out["rango_inicial"] = out["tol_superior"] - out["valor_nominal"]
    else:
        out["rango_inicial"] = out["tol_superior"] - out["tol_inferior"]

    # Campos requeridos
    out["valor_medio_nominal"] = out["valor_nominal"]
    out["valor_medio_real"]   = out["media_real"]
    out["sigma_w_inicial"]    = out["sigma"]
    out["cp_inicial"]         = out["cp"]
    out["cpk_inicial"]        = out.get("cpk_dir", np.nan)   # Cpd del ALS

    # Desviación inicial = ((Cp - Cpk)/Cp)
    out["desviacion_inicial"] = (out["cp_inicial"] - out["cpk_inicial"]) / out["cp_inicial"]

    # Cpk objetivo
    if cpk_objetivo:
        out["cpk_objetivo"] = out["variable"].map(cpk_objetivo).astype(float)
    else:
        out["cpk_objetivo"] = np.nan

    # Rango objetivo = 3 * sigma * Cpk objetivo
    out["rango_objetivo"] = 3 * out["sigma_w_inicial"] * out["cpk_objetivo"]

    cols = [
        "variable",
        "rango_inicial",
        "valor_medio_nominal",
        "valor_medio_real",
        "sigma_w_inicial",
        "cp_inicial",
        "cpk_inicial",
        "desviacion_inicial",
        "cpk_objetivo",
        "rango_objetivo",
    ]
    return out[cols]


In [ ]:
from google.colab import files
uploaded = files.upload()
PATH = list(uploaded.keys())[0]
PATH


In [ ]:

sub = read_als_summary(PATH)
display(sub.head())

rename = {
     "col_1": "Tiempo de ciclo [t4012]",
     "col_2": "Tiempo de inyección [t4018]",
     "col_3": "Tiempo de dosificación [t4015]",
     "col_4": "Cojín de masa [V4062]",
     "col_5": "Presión de conmutación [P4072]",
}

wide = tidy_als_summary(sub, rename_cols=rename)

objetivos = {
     "Tiempo de ciclo [t4012]": 5.0,
     "Tiempo de inyección [t4018]": 0.0,
     "Tiempo de dosificación [t4015]": 20.0,
     "Cojín de masa [V4062]": 6.0,
     "Presión de conmutación [P4072]": 0.0,
}

tabla = make_tracking_table(wide, cpk_objetivo=objetivos, half_range=False)

pd.set_option("display.float_format", lambda v: f"{v:,.3f}")
display(tabla)
